# Análisis para tienda online
## Objetivo:
Análisis de hipótesis para el aumento de ingresos
### Objetivos específicos:
* Priorizar las hipótesis
* Analizar el test A/B
    * Ingreso acumudalo por grupo
    * Número de pedidos por grupo
    * Ticket promedio por grupo y su diferencia relativa
    * Tasa de conversión para cada grupo
    * Identificar anomalías
    * Significancia estadística de la diferencia en la tasa de conversión y el ticket promedio para cada grupo


## Pasos a seguir
1. Priorizar las hipótesis
    * Aplicar framework ICE
    * Aplicar framework RICE

2. Análisis de test A/B
    * Limpieza de datos
    * Ingreso acumudalo por grupo
    * Número de pedidos por grupo
    * Ticket promedio por grupo y su diferencia relativa
    * Tasa de conversión para cada grupo
    * Identificar anomalías
    * Significancia estadística de la diferencia en la tasa de conversión y el ticket promedio para cada grupo

3. Toma de decisión sobre la prueba

## Diccionario de datos

**Hypotheses** (hipótesis)
* ``Hypotheses``: breves descripciones de las hipótesis
* ``Reach``: alcance del usuario, en una escala del uno a diez
* ``Impact``: impacto en los usuarios, en una escala del uno al diez
* ``Confidence``: confianza en la hipótesis, en una escala del uno al diez
* ``Effort``: los recursos necesarios para probar una hipótesis, en una escala del uno al diez. Cuanto mayor sea el valor Effort, más recursos requiere la prueba

**Orders** (pedidos realizados)
* ``transactionId``: identificador de pedido
* ``visitorId``: identificador del usuario que realizó el pedido
* ``date``: fecha del pedido
* ``revenue``: ingresos del pedido
* ``group``: el grupo del test A/B al que pertenece el usuario

**Visits** (registro de visitas)
* ``date``: la fecha
* ``group``: grupo de la prueba A/B
* ``visits``: el número de visitas en la fecha especificada en el grupo de pruebas A/B especificado

In [1]:
# importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from statsmodels.stats.proportion import proportions_ztest

# Priorizar hipótesis

In [2]:
# importar hipótesis
hypotheses = pd.read_csv('hypotheses_us.csv', sep=';')

# información general de 'hypotheses'
hypotheses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Effort      9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 492.0+ bytes


In [3]:
# nombres de columnas
hypotheses.columns = hypotheses.columns.str.lower()

In [4]:
# mostrar hipótesis
hypotheses

,hypothesis,reach,impact,confidence,effort
0,Add two new channels for attracting traffic. T...,3,10,8,6
1,Launch your own delivery service. This will sh...,2,5,4,10
2,Add product recommendation blocks to the store...,8,3,7,3
3,Change the category structure. This will incre...,8,3,3,8
4,Change the background color on the main page. ...,3,1,1,1
5,Add a customer review page. This will increase...,3,2,2,3
6,Show banners with current offers and sales on ...,5,3,8,3
7,Add a subscription form to all the main pages....,10,7,8,5
8,Launch a promotion that gives users discounts ...,1,9,9,5


In [5]:
# elaborar frameworks
# calcular 'ICE'
hypotheses['ICE'] = (hypotheses['impact'] * hypotheses['confidence']) / hypotheses['effort']

# calcular 'RICE'
hypotheses['RICE'] = (hypotheses['reach'] * hypotheses['impact'] * hypotheses['confidence']) / hypotheses['effort']

In [6]:
# prioridad de las hipótesis
print('Framework ICE', '\n', hypotheses[['hypothesis', 'ICE']].sort_values(by='ICE', ascending=False))
print()
print('Framework RICE', '\n', hypotheses[['hypothesis', 'RICE']].sort_values(by='RICE', ascending=False))

Framework ICE 
                                           hypothesis        ICE
8  Launch a promotion that gives users discounts ...  16.200000
0  Add two new channels for attracting traffic. T...  13.333333
7  Add a subscription form to all the main pages....  11.200000
6  Show banners with current offers and sales on ...   8.000000
2  Add product recommendation blocks to the store...   7.000000
1  Launch your own delivery service. This will sh...   2.000000
5  Add a customer review page. This will increase...   1.333333
3  Change the category structure. This will incre...   1.125000
4  Change the background color on the main page. ...   1.000000

Framework RICE 
                                           hypothesis   RICE
7  Add a subscription form to all the main pages....  112.0
2  Add product recommendation blocks to the store...   56.0
0  Add two new channels for attracting traffic. T...   40.0
6  Show banners with current offers and sales on ...   40.0
8  Launch a promotion that

### Comentario
Hay una diferencia en la priorización de las hipótesis dependiendo del framework empleado, esto se debe al parámetro 'reach' que emplea el 'RICE'. Sugiero priorizar las hipótesis tomando en cuenta ambos métodos para considerar tanto el impacto como el alcance.

Hipótesis recomendadas por orden de prioridad:
1) hipótesis 8: Add a subscription form to all the main pages. This will help you compile a mailing list
2) hipótesis 1: Add two new channels for attracting traffic. This will bring 30% more users
3) hipótesis 3: Add product recommendation blocks to the store's site. This will increase conversion and average purchase size